### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221004_scenario_projections.ipynb")

2022/09/28 21:49:04 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/model_dev_qc/vaccines/qc_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/28 21:49:10 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/28 21:49:13 INFO|-------------------------|{} Constructing vaccination projections
2022/09/28 21:49:13 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/28 21:49:15 INFO|----------------------|{} Prepping Model
2022/09/28 21:49:15 INFO|-----------------------|{} Prepping Model
2022/09/28 21:50:43 INFO|----------------------|{} Model prepped for fitting in 87.030457625 seconds.
2022/09/28 21:50:43 INFO|----------------------|{} Will fit 25 times
2022/09/28 21:51:52 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 69.22458974999999 seconds: {0: {'co': 0.830636682413169}, 14: {'co': 0.6982145962294486}, 28: {'co': 0.5695510909181374}, 42: {'co': 0.0027556383839568307}, 56: {'co': 0.6814732207342024}, 70: {'co': 0.8912739518691062}}
2022/09/28 21:51:52 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/28 21:

2022/09/28 21:56:48 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 9.966710416000012 seconds: {504: {'co': 0.7862044848399975}, 518: {'co': 0.7800183031631541}, 532: {'co': 0.7569129275753766}, 546: {'co': 0.7093499706644105}, 560: {'co': 0.7431991610520275}, 574: {'co': 0.7376795551483596}}
2022/09/28 21:56:48 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/28 21:57:04 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 14.393928332999963 seconds: {532: {'co': 0.7561820077522211}, 546: {'co': 0.7114199225292264}, 560: {'co': 0.7394392888827683}, 574: {'co': 0.7494247803082278}, 588: {'co': 0.7863432170313621}, 602: {'co': 0.7653246699243937}}
2022/09/28 21:57:04 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/28 21:57:17 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 11.266427249999992 seconds: 

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json"))

model_args = {
    'base_spec_id': 4067 #model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 1]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
               "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-30": vx_seed, "2022-10-20": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                            "attrs": {'age': '0-19', 'vacc': 'shot1'},
                            "mults": {"2020-01-01": 1,
                                      "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                      "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                      "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                      "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                      "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                      "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                      "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                            "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                    'hosp_reporting_frac': hrf,
                                    'tags': {'vx_seed': vx_seed,
                                             'vir_mult': vir_mult,
                                             'booster_mult': 1}})
            

In [5]:
# check how many scenarios there are
len(scenario_model_args)

4

In [6]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/28 22:08:05 INFO|============================================================
2022/09/28 22:08:05 INFO|============================================================
2022/09/28 22:08:05 INFO|============================================================
2022/09/28 22:08:05 INFO|============================================================
2022/09/28 22:08:09 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 1} Retrieving vaccinations data
2022/09/28 22:08:09 INFO|-----------|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 1} Retrieving vaccinations data
2022/09/28 22:08:09 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 1} Retrieving vaccinations data
2022/09/28 22:08:09 INFO|-----------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 1} Retrieving vaccinations data
2022/09/28 22:08:12 INFO|-----------|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 1} Constructing vaccination projections
2022/09/28 22:08:12 INFO|-----------|{'vx_seed': 1, 'vir

2022/09/28 22:10:42 INFO|--------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 1, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 14.707341041999996 seconds: {823: {'co': 0.8476310115056014}, 837: {'co': 0.8456664683083585}, 851: {'co': 0.8321996091062974}, 865: {'co': 0.8612338052100821}, 879: {'co': 0.8527866099671041}, 893: {'co': 0.8576454684451306}}
2022/09/28 22:10:42 INFO|---------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 1, 'fit_batch': '2'}: Running forward sim
2022/09/28 22:10:44 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 1, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 14.80623270800001 seconds: {823: {'co': 0.8476310115056014}, 837: {'co': 0.8456664683083585}, 851: {'co': 0.8321996091062974}, 865: {'co': 0.8612338052100821}, 879: {'co': 0.8527866099671041}, 893: {'co': 0.8576454684451306}}
2022/09/28 22:10:44 INFO|---------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 1, 'fit_batch': '2'}: Running forward sim

2022/09/28 22:11:24 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366815597537}, 14: {'co': 0.6982145991137699}, 28: {'co': 0.5650212471092002}, 42: {'co': 1.2408159663619181e-15}, 56: {'co': 0.6966527863556601}, 70: {'co': 0.833272726083201}, 84: {'co': 0.829624833881214}, 98: {'co': 0.8316925083025142}, 112: {'co': 0.8774896775135849}, 126: {'co': 0.8243044383057617}, 140: {'co': 0.7604527552865628}, 154: {'co': 0.6712880214899843}, 168: {'co': 0.8430551349085242}, 182: {'co': 0.7826363496218521}, 196: {'co': 0.8191511064245416}, 210: {'co': 0.7726071135217192}, 224: {'co': 0.7713215730133244}, 238: {'co': 0.7185439636873041}, 252: {'co': 0.7220971489408626}, 266: {'co': 0.6850776841685362}, 280: {'co': 0.7198336548294406}, 294: {'co': 0.7589635069294141}, 308: {'co': 0.8220311486521434}, 322: {'co': 0.8119730483051099}, 336: {'co': 0.7820760195219616}, 350: {'co': 0.7953719921669099}, 364: {'co': 0

### Run the report for each fit model

In [7]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir,
                           prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/09/28 22:23:29 INFO|============================================================
2022/09/28 22:23:29 INFO|--------|Solving model
2022/09/28 22:23:30 INFO|============================================================
2022/09/28 22:23:30 INFO|--------|Solving model
2022/09/28 22:23:31 INFO|============================================================
2022/09/28 22:23:31 INFO|--------|Solving model
2022/09/28 22:23:31 INFO|============================================================
2022/09/28 22:23:31 INFO|--------|Solving model
2022/09/28 22:23:34 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/28 22:23:34 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/28 22:23:35 WARNING|No artists with labels found to put in legend.  Note that artists 

In [8]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/28 22:55:59 INFO|Projecting
2022/09/28 22:55:59 INFO|
2022/09/28 22:56:21 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 22:56:23 INFO|
2022/09/28 22:56:45 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 22:56:47 INFO|
2022/09/28 22:57:09 INFO|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 22:57:11 INFO|
2022/09/28 22:57:33 INFO|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 1, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 22:57:35 INFO|Running reports
